In [1]:
from ast import literal_eval

import numpy as np
import pandas as pd
from sklearn import metrics

from constants import (SECTORS, SUBPILLARS_2D, SUBPILLARS_1D,
                       DEMOGRAPHIC_GROUPS, SPECIFIC_NEEDS_GROUPS,
                       AFFECTED_GROUPS, SEVERITY, RELIABILITY)

In [2]:
# this is weird a little bit, ask Selim about it
AFFECTED_GROUPS.append("Others")

In [3]:
tag_name_2_lbls = {
    "Sectors": {tag: i for i, tag in enumerate(SECTORS)},
    'Subpillars_2D': {tag:i for i, tag in enumerate(SUBPILLARS_2D)},
    'Subpillars_1D': {tag:i for i, tag in enumerate(SUBPILLARS_1D)},
    'Demographic_Groups': {tag:i for i, tag in enumerate(DEMOGRAPHIC_GROUPS)},
    'Affected_Groups': {tag:i for i, tag in enumerate(AFFECTED_GROUPS)},
    'Specific_Needs_Groups': {tag:i for i, tag in enumerate(SPECIFIC_NEEDS_GROUPS)},
    'Severity': {tag:i for i, tag in enumerate(SEVERITY)},
}
tag_names = [
    'Sectors', 'Subpillars_2D', 'Subpillars_1D', 'Demographic_Groups',
    'Affected_Groups', 'Specific_Needs_Groups', 'Severity'
]

In [7]:
df_03 = pd.read_csv("preds_0.3.csv")
df_031 = pd.read_csv("preds_0.3.1.csv")
df_02 = pd.read_csv("fb_processed_0.2.csv")

In [8]:
df_031.columns

Index(['Unnamed: 0', 'excerpt', 'sectors', 'subpillars_2d', 'subpillars_1d',
       'demographic_groups', 'affected_groups', 'specific_needs_groups'],
      dtype='object')

In [10]:
df_031.drop("Unnamed: 0", axis=1, inplace=True)

In [11]:
for col in df_02.columns[2:]:
    df_02[col] = df_02[col].apply(literal_eval)
##
for col in df_03.columns[1:]:
    df_03[col] = df_03[col].apply(literal_eval)
##
for col in df_031.columns[1:]:
    df_031[col] = df_031[col].apply(literal_eval)

In [14]:
df_03.columns = [
    'Entry', 'Sectors_Pred_0.3', 'Subpillars_2D_Pred_0.3',
    'Subpillars_1D_Pred_0.3', 'Demographic_Groups_Pred_0.3',
    'Affected_Groups_Pred_0.3', 'Specific_Needs_Groups_Pred_0.3',
    'Severity_Pred_0.3'
]
df_031.columns = [
    'Entry', 'Sectors_Pred_0.3.1', 'Subpillars_2D_Pred_0.3.1',
    'Subpillars_1D_Pred_0.3.1', 'Demographic_Groups_Pred_0.3.1',
    'Affected_Groups_Pred_0.3.1', 'Specific_Needs_Groups_Pred_0.3.1',
    #'Severity_Pred_0.3.1'
]

In [15]:
df = pd.merge(df_02, df_031, how="inner", on="Entry")

### Transform the Tags

In [16]:
# transform string labels into 2D binary matrix
def str_lbl_2_bmat(lbl_list, tag_name):
    lbl_count = len(tag_name_2_lbls[tag_name])
    lbl_mat = np.zeros((len(lbl_list) , lbl_count), dtype=np.int32)
    for i, lbls in enumerate(lbl_list):
        indices = [tag_name_2_lbls[tag_name][lbl] for lbl in lbls]
        lbl_mat[i, indices] = 1
    return lbl_mat

In [19]:
scores_v2 = {
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1Score": [],
    "HammingLoss": [],
    "ZeroOneLoss": []
}
scores_v3 = {
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1Score": [],
    "HammingLoss": [],
    "ZeroOneLoss": []
}
scores_v2_v3 = {
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1Score": [],
    "HammingLoss": [],
    "ZeroOneLoss": []
}
for tag_name in tag_names[:-1]:
    y_true = str_lbl_2_bmat(df[f"{tag_name}_GT"], tag_name)
    y_pred_v2 = str_lbl_2_bmat(df[f"{tag_name}_Pred_0.2"], tag_name)
    y_pred_v3 = str_lbl_2_bmat(df[f"{tag_name}_Pred_0.3.1"], tag_name)
    # accuracy
    acc_score_v2 = metrics.accuracy_score(y_true, y_pred_v2)
    acc_score_v3 = metrics.accuracy_score(y_true, y_pred_v3)
    scores_v2["Accuracy"].append(acc_score_v2)
    scores_v3["Accuracy"].append(acc_score_v3)
    scores_v2_v3["Accuracy"].append(f"{acc_score_v2:.2f} | {acc_score_v3:.2f}")
    # precision
    precision_score_v2 = metrics.precision_score(y_true,
                                                 y_pred_v2,
                                                 average="micro")
    precision_score_v3 = metrics.precision_score(y_true,
                                                 y_pred_v3,
                                                 average="micro")
    scores_v2["Precision"].append(precision_score_v2)
    scores_v3["Precision"].append(precision_score_v3)
    scores_v2_v3["Precision"].append(
        f"{precision_score_v2:.2f} | {precision_score_v3:.2f}")
    # recall
    recall_score_v2 = metrics.recall_score(y_true, y_pred_v2, average="micro")
    recall_score_v3 = metrics.recall_score(y_true, y_pred_v3, average="micro")
    scores_v2["Recall"].append(recall_score_v2)
    scores_v3["Recall"].append(recall_score_v3)
    scores_v2_v3["Recall"].append(
        f"{recall_score_v2:.2f} | {recall_score_v3:.2f}")
    # f1 score
    f1_score_v2 = metrics.f1_score(y_true, y_pred_v2, average="micro")
    f1_score_v3 = metrics.f1_score(y_true, y_pred_v3, average="micro")
    scores_v2["F1Score"].append(f1_score_v2)
    scores_v3["F1Score"].append(f1_score_v3)
    scores_v2_v3["F1Score"].append(f"{f1_score_v2:.2f} | {f1_score_v3:.2f}")
    # hamming loss
    hamming_loss_v2 = metrics.hamming_loss(y_true, y_pred_v2)
    hamming_loss_v3 = metrics.hamming_loss(y_true, y_pred_v3)
    scores_v2["HammingLoss"].append(hamming_loss_v2)
    scores_v3["HammingLoss"].append(hamming_loss_v3)
    scores_v2_v3["HammingLoss"].append(
        f"{hamming_loss_v2:.2f} | {hamming_loss_v3:.2f}")
    # zero/one loss
    zero_one_loss_v2 = metrics.zero_one_loss(y_true, y_pred_v2)
    zero_one_loss_v3 = metrics.zero_one_loss(y_true, y_pred_v3)
    scores_v2["ZeroOneLoss"].append(zero_one_loss_v2)
    scores_v3["ZeroOneLoss"].append(zero_one_loss_v3)
    scores_v2_v3["ZeroOneLoss"].append(
        f"{zero_one_loss_v2:.2f} | {zero_one_loss_v3:.2f}")

/home/abdullah/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
scores_v2_df = pd.DataFrame.from_dict(scores_v2)
scores_v2_df.index = tag_names[:-1]
##
scores_v3_df = pd.DataFrame.from_dict(scores_v3)
scores_v3_df.index = tag_names[:-1]
#
scores_v2_v3_df = pd.DataFrame.from_dict(scores_v2_v3)
scores_v2_v3_df.index = tag_names[:-1]


In [22]:
pd.options.display.float_format = '{:,.2f}'.format

In [23]:
scores_v2_df

,Accuracy,Precision,Recall,F1Score,HammingLoss,ZeroOneLoss
Sectors,0.88,0.94,0.92,0.93,0.02,0.12
Subpillars_2D,0.57,0.90,0.58,0.71,0.04,0.43
Subpillars_1D,0.49,0.36,0.86,0.51,0.02,0.51
Demographic_Groups,0.22,0.18,0.91,0.30,0.09,0.78
Affected_Groups,0.93,0.00,0.00,0.00,0.00,0.07
Specific_Needs_Groups,0.53,0.10,0.88,0.18,0.05,0.47


In [24]:
scores_v3_df

,Accuracy,Precision,Recall,F1Score,HammingLoss,ZeroOneLoss
Sectors,0.54,0.95,0.48,0.64,0.07,0.46
Subpillars_2D,0.28,0.69,0.37,0.48,0.06,0.72
Subpillars_1D,0.68,0.56,0.14,0.22,0.01,0.32
Demographic_Groups,0.80,0.15,0.22,0.18,0.04,0.20
Affected_Groups,0.33,0.02,0.38,0.03,0.10,0.67
Specific_Needs_Groups,0.94,0.57,0.50,0.53,0.01,0.06


In [25]:
scores_v2_v3_df

,Accuracy,Precision,Recall,F1Score,HammingLoss,ZeroOneLoss
Sectors,0.88 | 0.54,0.94 | 0.95,0.92 | 0.48,0.93 | 0.64,0.02 | 0.07,0.12 | 0.46
Subpillars_2D,0.57 | 0.28,0.90 | 0.69,0.58 | 0.37,0.71 | 0.48,0.04 | 0.06,0.43 | 0.72
Subpillars_1D,0.49 | 0.68,0.36 | 0.56,0.86 | 0.14,0.51 | 0.22,0.02 | 0.01,0.51 | 0.32
Demographic_Groups,0.22 | 0.80,0.18 | 0.15,0.91 | 0.22,0.30 | 0.18,0.09 | 0.04,0.78 | 0.20
Affected_Groups,0.93 | 0.33,0.00 | 0.02,0.00 | 0.38,0.00 | 0.03,0.00 | 0.10,0.07 | 0.67
Specific_Needs_Groups,0.53 | 0.94,0.10 | 0.57,0.88 | 0.50,0.18 | 0.53,0.05 | 0.01,0.47 | 0.06
